In [14]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from math import factorial
from datetime import datetime
import os
from matplotlib.colors import ListedColormap
import lmfit
from tqdm.notebook import tqdm
from resram_core import load_input, g,A, R, cross_sections, run_save, resram_data, raman_residual, param_init


In [15]:

fit_obj = load_input() # Initialize the 'fit_obj' object of class 'load_input' .Load from input files in root dir

fit_switch = np.ones(len(fit_obj.delta)+7)
fit_switch[len(fit_obj.delta)] = 1  # fit gamma?
fit_switch[len(fit_obj.delta)+1] = 1  # fit M?
fit_switch[len(fit_obj.delta)+2] = 1  # fit theta?
fit_switch[len(fit_obj.delta)+3] = 1  # fit kappa?
fit_switch[len(fit_obj.delta)+5] = 0  # fit E0?

correlation_list = []   # Initialize an empty list to store uvvis correlation values
sigma_list = [] # Initialize an empty list to store raman prof sigma values
loss_list = []  # Initialize an empty list to store total loss values
params_lmfit = param_init(fit_obj,fit_switch)   # Initialize parameters for fitting using 'param_init' function
# params_lmfit.add('gamma',value = 600,min = 400, max = 900)
# params_lmfit.add('transition_length',value = 1.83,min = 1.5, max = 2.3)
for name in params_lmfit.keys():
    print(params_lmfit[name])
raman_residual(params_lmfit)   # Calculate initial residuals for Raman spectroscopy
fit_kws = dict(tol=1e-7)    # Set fitting tolerance
max_nfev = 10000    # Set maximum number of function evaluations

# Initialize tqdm progress bar for fitting process
with tqdm(total=max_nfev, desc="Fitting progress") as pbar:
    def update_progress(params, iteration, resid):
        # Update tqdm progress bar with fitting information
        global correlation, total_sigma
        pbar.update(1)
        pbar.set_description(
            f"Iteration {iteration}, Total Residual: {resid:.7f}, Abs_corr: {correlation:.7f}, raman_sigma: {total_sigma:.7f}, delta: {fit_obj.delta}, M: {fit_obj.M}, Theta: {fit_obj.theta}, Kappa: {fit_obj.k}, E0: {fit_obj.E0}")
     # Perform the fitting using lmfit.minimize function
    result = lmfit.minimize(raman_residual, params_lmfit, method='cobyla', **fit_kws,
                            max_nfev=max_nfev, iter_cb=update_progress)  # max_nfev = 10000000, **fit_kws

print(lmfit.fit_report(result))
delta_fit = fit_obj.delta.copy()
output = run_save(fit_obj)  # save all parameters and results to a new folder
output.plot()
np.savetxt(current_time_str+"_data/corr_list.dat", correlation_list)
np.savetxt(current_time_str+"_data/sigma_list.dat", sigma_list)
np.savetxt(current_time_str+"_data/loss_list", loss_list)
#np.savetxt(current_time_str+"_data/delta_fit.dat",delta_fit,delimiter = "\t") #save deltas. already saved with run_save()
plt.plot(np.arange(len(correlation_list)), np.array(
    correlation_list)*50, label='correlation*50')
plt.plot(np.arange(len(sigma_list)), sigma_list, label='sigma')
plt.plot(np.arange(len(loss_list)), loss_list, label='loss')
plt.legend(loc='best')
plt.show()


<Parameter 'delta0', value=0.7748604270679489, bounds=[0.0:1.0]>
<Parameter 'delta1', value=0.6409960961625931, bounds=[0.0:1.0]>
<Parameter 'delta2', value=0.5872857740485251, bounds=[0.0:1.0]>
<Parameter 'delta3', value=0.20719247724224388, bounds=[0.0:1.0]>
<Parameter 'delta4', value=0.1607544533791192, bounds=[0.0:1.0]>
<Parameter 'delta5', value=0.3893313623373211, bounds=[0.0:1.0]>
<Parameter 'delta6', value=0.12074773193374017, bounds=[0.0:1.0]>
<Parameter 'delta7', value=0.11603393193686784, bounds=[0.0:1.0]>
<Parameter 'delta8', value=0.30537623378500867, bounds=[0.0:1.0]>
<Parameter 'delta9', value=0.15836620976768084, bounds=[0.0:1.0]>
<Parameter 'delta10', value=0.20742166396327966, bounds=[0.0:1.0]>
<Parameter 'delta11', value=0.060015796848877534, bounds=[0.0:1.0]>
<Parameter 'delta12', value=0.08745576005571493, bounds=[0.0:1.0]>
<Parameter 'delta13', value=0.0968567706185624, bounds=[0.0:1.0]>
<Parameter 'delta14', value=0.0916268892643442, bounds=[0.0:1.0]>
<Parameter 

NameError: name 'fit_obj' is not defined